In [1]:
import sys
sys.path.append('Bin/')
import Lib
import re
from urllib import parse
from googletrans import Translator

translator = Translator()
libray = Lib.CppAssignment()
news = libray.Newspaper()

def k2u(text):
    url = text.replace('/', '|')
    print(url)
    url = parse.urlparse(f'http://www.exeam.org?examParam1=value1&examParam2={text}') 
    query = parse.parse_qs(url.query)
    data = parse.urlencode(query,  doseq=True)
    encording = data.replace('examParam1=value1&examParam2=', '')
    return encording

def cleanText(readData):
 
    #텍스트에 포함되어 있는 특수 문자 제거
 
    text = re.sub("[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]", '', readData)
 
    return text

Papago Client : HuA_wT4ehgG82Bo3mWct
Papago Secret : YJ8zMly9WN


In [2]:
from flask import Flask
import re

app = Flask(__name__)


@app.route('/')
def hello_world():
    return 'Hello, World!'


@app.route('/input/<data>')  # URL뒤에 <>을 이용해 가변 경로를 적는다
def hello_user(data):
    data = data.replace('%3A', ':')
    #data = data.replace('%3F', '?')
    url = data.replace('|', '/')
    print("URL :", url)
    try:
        data = news.Newspaper(url=url, language="ko")
        if len(data) == 0:
            data = news.Newspaper(url=url, language="en")
            if len(data) == 0:
                data = news.Newspaper(url=url, language="ja")
                if len(data) == 0:
                    data = news.Newspaper(url=url, language="zh")
                    print("SUMM :",data[0:50])
                    getSource = data[0:50]
                    source = libray.Papago(text=getSource).GetLangage()
                    print("SOURCE :", source)
                else:
                    print("SUMM :",data[0:50])
                    getSource = data[0:50]
                    source = libray.Papago(text=getSource).GetLangage()
                    print("SOURCE :", source)
            else:
                print("SUMM :",data[0:50])
                getSource = data[0:50]
                source = libray.Papago(text=getSource).GetLangage()
                print("SOURCE :", source)
        else:
            print("SUMM :",data[0:50])
            getSource = data[0:50]
            source = libray.Papago(text=getSource).GetLangage()
            print("SOURCE :", source)
    except:
        return f"[ERROR] CANT SUMMARY PLEASE CHECK THIS PAGE : {url}"
    
    
    try:
        if source != 'ko':
            print("[use google translator]")
            text = translator.translate(data, dest='ko').text
            print("ori :", data)
            print("tra :", text)
            sentence_data = libray.KoNLPy(text).sentence()
            data_pn_score = []
            obj = [0]
            for i in range(len(sentence_data)):
                if len(sentence_data[i]) < 21:
                    score = 0
                    tf = 0
                else:
                    _, score, tf = libray.model().text(sentence_data[i])
                    data_pn_score.append(libray.model().text(sentence_data[i])[1:])
                    #tf == 1 부정

                if tf == 1:
                    obj.append(-score)
                else:
                    obj.append(score)

            print("\n총 점수 : ",round(int(sum(obj)) / len(obj), 1))

            for i in range(len(sentence_data)):
                print(obj[i+1], sentence_data[i])

            if int(sum(obj)) < 0:
                tf = "Negative"
            elif(int(sum(obj)) == 0):
                tf = "neutral"
            else:
                tf = "Positive"


            a = f"""[{str(tf)} : {str(round(int(sum(obj)) / len(obj), 1))}] ************** {str(text)}"""
            return a

        else:
            print("in else")
            print(data)
            sentence_data = libray.KoNLPy(data).sentence()
            data_pn_score = []
            obj = [0]
            print(sentence_data)
            print("go for")
            for i in range(len(sentence_data)):
                print(len(sentence_data[i]) < 21)
                if len(sentence_data[i]) < 21:
                    score = 0
                    tf = 0
                else:
                    _, score, tf = libray.model().text(sentence_data[i])
                    
                    data_pn_score.append(libray.model().text(sentence_data[i])[1:])
                    #tf == 1 부정
                if tf == 1:
                    obj.append(-score)
                else:
                    obj.append(score)
            print("\n총 점수 : ",int(sum(obj)))

            for i in range(len(sentence_data)):
                print(obj[i+1], sentence_data[i])


            if int(sum(obj)) < 0:
                tf = "Negative"
            elif(int(sum(obj)) == 0):
                tf = "neutral"
            else:
                tf = "Positive"


            a = f"""[{str(tf)} : {str(round(int(sum(obj)) / len(obj), 1))}] ************** {str(data)}"""
            return a
    except:
        return data
    
if __name__ == "__main__":
    app.run()


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [23/Dec/2019 20:38:22] "GET / HTTP/1.1" 200 -


URL : https://news.v.daum.net/v/20191223201502676
SUMM : "국회법 결과 무제한 토론 적합하지 않아" [서울=뉴시스]김명원 기자 = 문희상 국회의장.
SOURCE : ko
in else
"국회법 결과 무제한 토론 적합하지 않아" [서울=뉴시스]김명원 기자 = 문희상 국회의장.문 의장은 23일 오후 7시57분께 개의된 본회의에서 "국회 임시회 회기 결정의 건을 상정한다"며 "이 안건에 대해 심재철 의원 등 108인으로부터 무제한 토론 요구가 제출되었지만 국회법 결과 회기 결정의 건은 무제한 토론이 적합하지 않다는 결론에 이르렀다"고 밝혔다.
['" 국회 법 결과 무제한 토론 적합하지 않아" [ 서울= 뉴 시스] 김명원 기자 = 문 희상 국회의장. 문 의장은 23일 오후 7시 57분께 개의된 본회의에서 " 국회 임시회 회기 결정의 건을 상정한다 "며 " 이 안건에 대해 심재 철 의원 등 108 인으로부터 무제한 토론 요구가 제출되었지만 국회법 결과 회기 결정의 건은 무제한 토론이 적합하지 않다는 결론에 이 르 렀 다" 고 밝혔다.']
go for
False


127.0.0.1 - - [23/Dec/2019 20:38:45] "GET /input/https%3A%7C%7Cnews.v.daum.net%7Cv%7C20191223201502676 HTTP/1.1" 200 -



총 점수 :  -100
-100.0 " 국회 법 결과 무제한 토론 적합하지 않아" [ 서울= 뉴 시스] 김명원 기자 = 문 희상 국회의장. 문 의장은 23일 오후 7시 57분께 개의된 본회의에서 " 국회 임시회 회기 결정의 건을 상정한다 "며 " 이 안건에 대해 심재 철 의원 등 108 인으로부터 무제한 토론 요구가 제출되었지만 국회법 결과 회기 결정의 건은 무제한 토론이 적합하지 않다는 결론에 이 르 렀 다" 고 밝혔다.
URL : https://news.v.daum.net/v/20191223201756721
SUMM : 심재철 원내대표 등 일부 의원들은 오후 6시50분께 국회의장실로 이동해 면담을 요청하며 본
SOURCE : ko
in else
심재철 원내대표 등 일부 의원들은 오후 6시50분께 국회의장실로 이동해 면담을 요청하며 본회의 개의 시도에 강력 항의했다.[서울=뉴시스] 고승민 기자 = 자유한국당 심재철 원내대표와 이주영 국회부의장 등 의원들이 23일 서울 여의도 국회에서 예산부수법안과 공수처법, 유아교육법 개정안 등을 안건으로 제372회국회(임시회) 제1차 본회의가 열리기 전 문희상 의장실을 찾아 면담을 요구하고 있다.[서울=뉴시스] 고승민 기자 = 자유한국당 심재철 원내대표와 이주영 국회부의장 등 의원들이 23일 서울 여의도 국회에서 예산부수법안과 공수처법, 유아교육법 개정안 등을 안건으로 제372회국회(임시회) 제1차 본회의가 열리기 전 문희상 의장실을 찾아 면담을 요구하고 있다.이들은 한시간 넘게 의장실 앞에 서서 문을 두드리며 의장과의 면담을 요청했으나 결국 오후 7시56분께 문 의장이 뒷문으로 빠져나가 본회의장으로 들어가면서 면담은 무산됐다.
['심재 철 원내 대표 등 일부 의원들은 오후 6시 50분께 국회의 장실로 이동해 면담을 요청하며 본회의 개의 시도에 강력 항의했다.', '[ 서울= 뉴 시스] 고승 민 기자 = 자유 한국당 심재 철 원내 대표와 이주 영 국회 부의장 등 의원들이 23일 서울 여의도 국회에서 예산 부수 법안과 공수

...


총 점수 :  85.3
100.0 숫자는 지금 너무 거대하고, 피해 그리 좋은, 그 권한 샷으로 야생 조류 부여 된 - 먹을, BBC 의 엠 마 제인 Kirby.In 대부분의 상황을 보고는 농부의 분야에 걸쳐 부정 기선으로 현명하지 못한 것으로 간주 될 것이다 위장 기어를 입고 꼬마 작은 시간에 무기를 운반하는 동안, 하지만 우리가 들을 수 있는 우리가 바닷물 호수에 접근 스티브 로저스 행진 자신 있게 스트 롬 니스, 오크 니의 냉동 된 잔디를 가로 질러, 그의 전투 부츠에서 스냅 블레이드 및 얼음 웅덩이 분열 젖 빛 주민 거위의 경적을 울리고 기절. " 그들은 짧은 시간에 할 손상하는 것은 믿을 수 없다" 고 dawn.Islanders 의 첫 번째 빛에 있었던 자신의 휘파람 기러기를 호출 할 준비를 오크 니가, 휴일 슈팅 실행 스티브는 말한다 2012 년부터 조류의 세트 수를 제어 할 수 있지만, 거위 방목의 경제 및 환경에 미치는 영향은 스코 틀 랜드의 자연 유산 (SNH) 이, 몸이 스코 틀 랜드의 Wi 보호로 기소 것을 crofters 지금 너무 심각 ldlife, 심지어 자원 봉사 shooters.The 총은 샷의 한바탕 소동을 균열 스티브 플라스틱 미끼의 에서 밖으로 돌진하고, 타락 새를 검색하기 위해 자신의 래브라도, 잭, 인종 아웃 출시 모집, 이번 여름에 더 큰 도태에의 지지를 준 의 casualties.Image 캡션 검색 기러기는 알 리 스터 DonaldsonGraham 네 빌, 스코 틀 랜드의 자연 유산의 북부 섬과 북쪽 고원 지역 관리자와 같은 농부들에게 중요한 문제이며, 여전히 자연 보호의 의무를 이행하면서 거위 대한 SNH의 정책이 지속 가능한 솔루션을 찾는 것에 대해 말한다 그들은 호수에 떠 거위의 다음 타래를 기다리고 분야에서 미끼를 향해 비행으로 온도가 짓궂은 -3C 위쪽으로 급증했다 .IN
100.0 스티브의 숨기기, 저격수는 끓는 달콤한 커피의 술병 라운드 통과 스티브는 자신의 휘파람 소리. 롭의 슈팅 게임 중 하나가 웃음

Building prefix dict from e:\anaconda\envs\nasa\lib\site-packages\jieba\dict.txt ...
Dumping model to file cache C:\Users\Seungup\AppData\Local\Temp\jieba.cache
Loading model cost 1.309166431427002 seconds.
Prefix dict has been built succesfully.


SUMM : 习近平强调，双方要以实现高质量融合发展为目标，进一步挖掘潜力，提升务实合作层次和水平。双方可以推动共
SOURCE : zh-CN
[use google translator]
ori : 习近平强调，双方要以实现高质量融合发展为目标，进一步挖掘潜力，提升务实合作层次和水平。双方可以推动共建“一带一路”倡议同韩方发展战略规划对接进一步早见实效、早结硕果，加快双边自由贸易协定第二阶段谈判，深化创新研发合作，更好实现优势互补和成果共享。要用好中韩人文交流促进委员会平台，开展好教育、体育、媒体、青少年、地方等领域交流，不断增进两国人民相互了解和友好感情。应该加强执法安全合作，共同打击跨境犯罪，维护两国社会安定和谐。
tra : 시진핑는 목표로 개발 고품질의 통합을 달성하기 위해 양측이 더 협력과 실용적인 수준의 수준을 향상시킬 수있는 잠재력을 탐구하는 것을 강조했다. 양측은 홍보와 전략 기획 하야미 효과적인 조기 곰 과일의 발전에 한국에 주도권 도킹 "길을 따라"빌드, 혁신적인 R & D 협력, 상호 보완적인 장점과 결과를 더 잘 공유의 개발을 양자 간 자유 무역 협정 (FTA) 협상의 두 번째 단계를 가속화 할 수 있습니다. 교환 Hanren 원 총리는 의회에서보다 효율적으로 사용하기 위해 프로모션 플랫폼은 좋은 교육, 스포츠, 미디어, 청소년, 등의 교류 등 지역의 영역을 수행하고, 지속적으로 양국 국민 감정 사이의 상호 이해와 우정을 강화하기 위해. 법 집행 및 안보 협력과 공동으로 전투 국경 범죄를 강화해야, 두 나라의 사회 안정과 조화를 유지합니다.


127.0.0.1 - - [23/Dec/2019 20:40:26] "GET /input/http%3A%7C%7Cnews.cctv.com%7C2019%7C12%7C23%7CARTIOMIfXBfRsEyRSLqykKqG191223.shtml%3Fspm%3DC96370.PsikHJQ1ICOX.Em32AuyOHUeL.1 HTTP/1.1" 200 -



총 점수 :  -25.0
-100.0 시진 핑 는 목표로 개발 고품질의 통합을 달성하기 위해 양측이 더 협력과 실용적인 수준의 수준을 향상시킬 수 있는 잠재력을 탐구하는 것을 강조했다.
-100.0 양측은 홍보와 전략 기획 하야 미 효과 적인 조기 곰 과일의 발전에 한국에 주도권 도킹 " 길을 따라" 빌드, 혁신적인 R & D 협력, 상호 보완적인 장점과 결과를 더 잘 공유의 개발을 양자 간 자유 무역 협정 (FTA) 협상의 두 번째 단계를 가속화 할 수 있습니다.
100.0 교환 Hanren 원 총리는 의회에서 보다 효율적으로 사용하기 위해 프로 모션 플랫폼은 좋은 교육, 스포츠, 미디어, 청소년, 등의 교류 등 지역의 영역을 수행하고, 지속적으로 양국 국민 감정 사이의 상호 이해와 우정을 강화하기 위해. 법 집행 및 안보 협력과 공동으로 전투 국경 범죄를 강화해야, 두 나라의 사회 안정과 조화를 유지합니다.
